In [1]:
''' Implicit Trust based recommendation system
    developed by Priyanka Singhal
'''
import itertools
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# STEP 1: Load the Epinions Dataset. Pre-process dataset to select only relevant columns - item, user, stars

In [2]:
#Step 1: Load the dataset
file_path = 'epinions_with_quotes.json'
epinion_data = pd.read_json(file_path, lines=True)
epinion_data.head(3)
print('Rows:', epinion_data.shape[0], '; Columns:', epinion_data.shape[1], '\n')

# Select relevant columns (item, user, stars)
epinion_data_slim = epinion_data[['item', 'user', 'stars']]
print("First 4 rows of relevant columns only ")
print(epinion_data_slim.head(4))

Rows: 188477 ; Columns: 6 

First 4 rows of relevant columns only 
                                                item        user  stars
0                 Minolta_QMS_PagePro_1250E_Printers      fgb59h      4
1  Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...    bucho_ky      2
2  Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...     redp944      4
3              pr-Durabrand_CD-85_Personal_CD_Player  spongebag7      4


# STEP 2 - Pre-process dataset further to select training and test splits. We do an 80:20 split for training/test. 

In [3]:
# Convert data to tuples of (item, user, rating) triplets
dataset_tuples = [tuple(x) for x in epinion_data_slim.values]
print("Total number of tuples before dataset split: {}".format(len(dataset_tuples)))

# Randomly shuffle dataset with fixed seed. Then split into train/test using 80:20 ratio
random.Random(4).shuffle(dataset_tuples)
train_data = dataset_tuples[:152000]
test_data = dataset_tuples[152000:]

# Print length of train and test dataset
print("Length of training set: {}".format(len(train_data)))
print("Length of test set: {}".format(len(test_data)))

Total number of tuples before dataset split: 188477
Length of training set: 152000
Length of test set: 36477


# STEP 3 - Construct user2item and item2user dictionaries from training data. Also, retain rating for each value in both dictionaries

In [4]:
# Construct user2item from training data
user2item = {}
for data_tuple in train_data:
    if data_tuple[1] in user2item:
        user2item[data_tuple[1]].add((data_tuple[0], data_tuple[2]))
    else:
        itemrating = set()
        itemrating.add((data_tuple[0], data_tuple[2]))
        user2item[data_tuple[1]] = itemrating

# Construct item2user from training data
item2user = {}
for data_tuple in train_data:
    if data_tuple[0] in item2user:
        item2user[data_tuple[0]].add((data_tuple[1], data_tuple[2]))
    else:
        userrating = set()
        userrating.add((data_tuple[1], data_tuple[2]))
        item2user[data_tuple[0]] = userrating
        
print("Total number of entries in user2item: {}".format(len(user2item)))
print("Total number of entries in item2user: {}".format(len(item2user)))
print("Printing 5 random entries in user2item.. ")
for i, val in enumerate(itertools.islice(user2item, 5)):
    print(val)
    print(user2item[val])

Total number of entries in user2item: 98562
Total number of entries in item2user: 37417
Printing 5 random entries in user2item.. 
consumerdude
{('pr-Digital_Cameras-Sony_DSC-P8_Digital_Camera', 1), ('cmhd-Accessories-All-Wireless_Optical_Mini_Mouse_PAUM005U', 1), ('Sony_DSC_P7_Digital_Camera_Digital_Cameras', 5), ('logitech-wireless-touch-keyboard-k400-920003116', 5), ('Sony_Cyber_shot_DSC_T200_Digital_Camera_SNY13003', 4), ('pr-Dell_Axim_X5_400_MHz_Personal_Organizer', 3), ('pr-RIM_BlackBerry_7100', 5), ('nikon-coolpix-p310-light-field-camera', 5)}
ttdsgnr
{('auto_Make-1987_Suzuki_Samurai', 3), ('auto_Make-1998_Honda_CR_V', 5), ('auto_Make-1990_Honda_Civic_2_Door', 5)}
darobin
{('elec_Portable_Audio-MD_Walkman_Sony_MZ_R-Sony_MZ-B3', 5), ('elec_Portable_Audio-MD_Walkman_Sony_MZ_R-Sony_MZ-R5ST', 5), ('Dell_Adamo_XPS_Laptop_with_Intel_Core_2_Duo_Processor_Silver_AX3601GSL_PC_Notebook', 5)}
Chris_Billings
{('LEGO_Collectible_Minifigure_Zombie_epi', 4), ('Hasbro_Star_Wars_The_Clone_Wars_R2

# STEP 4 - Approach 1 - Implicit trust. We will now write methods to compute implicit trust between users.

In [5]:
# Fixed hyperparameters
alpha = 0.4
beta = 0.6
sigma = 0.2


def compute_ji(u1, u2, user2item, item2user):
    """
    Compute Jaccard-index for user1 and user2
    """
    # Get all items for u1    
    u1_items = [tup[0] for tup in user2item[u1]]
    
    # For all u1 items, get all users wbo have rated it
    u1_nbs = set()
    for u1_item in u1_items:
        # Get all users who have rated item
        users = [tup[0] for tup in item2user[u1_item]]
        # Extend u1_nbs
        u1_nbs.update(users)
    
    # Get all items for u2    
    u2_items = [tup[0] for tup in user2item[u2]]
    
    # For all u2 items, get all users wbo have rated it
    u2_nbs = set()
    for u2_item in u2_items:
        # Get all users who have rated item
        users = [tup[0] for tup in item2user[u2_item]]
        # Extend u1_nbs
        u2_nbs.update(users)    

    common_users = u1_nbs.intersection(u2_nbs)
    union_users = u1_nbs.union(u2_nbs)
   
    # Return ji
    return len(common_users)/ (1.0 * len(union_users))
    
def calc_d(i1, sigma, item2user):
    """
    Calculate item popularity index D(i) for item i
    """
    # Get number of user who have rated item
    degree_i1 = len(item2user[i1])
    
    # return D(i)
    return (2.0 / (1 + np.exp(2**sigma - degree_i1**sigma))) - 1.0

# Unit test for compute_ji. Expected output is 0.2352941176
print(compute_ji('pyros7', 'tarsyn', user2item, item2user))

# Unit test for calc_d. Expected output is 0.08519773513
print(calc_d('pr-Sony_KV_36FS12__Standard_Televisions', sigma, item2user))

# Unit test for calc_d. Expected output is 0.266655586
print(calc_d('auto_Make-2001_Pontiac_Firebird', sigma, item2user))

0.23529411764705882
0.08519773513040674
0.26665558577993353


In [6]:
def calc_common_item_popularity(u1, u2, sigma, user2item, item2user):
    """
    Calculate common item popularity for user1 and user2
    """
    # Get all items for u1
    u1_items = set([tup[0] for tup in user2item[u1]])

    # Get all items for u2
    u2_items = set([tup[0] for tup in user2item[u2]])

    # For common items, calculate D(i)
    common_items = u1_items.intersection(u2_items)
    if not common_items or len(common_items) == 0:
        return 0.0
    
    # Handling edge cases
    sum_d = 0.0
    for common_item in common_items:
        sum_d = sum_d + calc_d(common_item, sigma, item2user)
    
    return 1.0  - (sum_d)/(len(common_items))

# Unit test for calc_common_item_popularity. Expected output is 0.9148022648695933
print(calc_common_item_popularity('pyros7', 'tarsyn', sigma, user2item, item2user))

0.9148022648695933


In [7]:
def compute_implicit_trust(u1, u2, alpha, beta, sigma, user2item, item2user):
    # Step 1 - Compute Jaccard Index
    ji = compute_ji(u1, u2, user2item, item2user)
    #  print("ji ".format(ji))
    # Step 2 - Compute common similarity
    com_sim = calc_common_item_popularity(u1, u2, sigma, user2item, item2user)
    
    # Step 3 - Combine both
    return alpha * ji + beta * com_sim

# Unit test for compute_implicit_trust. Expected output is 0.642999006
print(compute_implicit_trust('pyros7', 'tarsyn', alpha, beta, sigma, user2item, item2user))

0.6429990059805795


# STEP 5 - Before predicting ratings on the test set, define common methods for error/result calculation

In [8]:
# Methods for Mean Absolute Error (MAE) and Mean Squared Error (MSE)
def calculate_mae(result_set):
    mae = 0.0
    for result in result_set:
        mae = mae + np.abs(result[0] - result[1])
    return mae / len(result_set)

def calculate_rmse(result_set):
    rmse = 0.0
    for result in result_set:
        rmse = rmse + (result[0] - result[1]) * (result[0] - result[1])
    return rmse / len(result_set)

def run_inference_on_test_set(test_data, user2item, item2user, inference_method,alpha, beta, sigma):
    """
    Run infereence on test set, based on inference_method (function parameter)
    """
    # Create counters for actual inference, user-cold start and item-cold start
    actual_inference_count = 0
    user_cold_start_cnt = 0
    item_cold_start_cnt = 0
    
    # Run inference on test set
    result_set = []
    for idx, test_sample in enumerate(test_data): 
        if not test_sample[1] in user2item:
            user_cold_start_cnt += 1
            continue
        if not test_sample[0] in item2user:
            item_cold_start_cnt += 1
            continue

        ground_truth_rating = test_sample[2]
        predicted_rating = inference_method(test_sample[1], test_sample[0], alpha, beta, sigma, user2item, item2user)
        if predicted_rating == 0.0:
            # Skip since user might not have anything in common with other raters of item
            continue        
        result_set.append((ground_truth_rating, predicted_rating))
        actual_inference_count += 1
    # Print stats
    print(actual_inference_count)
    print("Item cold start count : {}".format(item_cold_start_cnt))
    print("User cold start count : {}".format(user_cold_start_cnt))

    print("Coverage : {}".format(actual_inference_count / (1.0 * len(test_data)) ))
    print("Mean Absolute Error : {}".format(calculate_mae(result_set)))
    print("RMSE : {}".format(calculate_rmse(result_set)))

# STEP 6 - Different methods to predict test ratings

# STEP 6.1 - Method 1 - Simple trust-based mean


In [9]:
def calc_trust_based_mean_rating(u, i, alpha, beta, sigma, user2item, item2user):
    """
    Calculate trust-based mean rating on test set
    """
    # Get all users who have rated item
    users_and_ratings = [tup for tup in item2user[i]]
    
    # For each user, calculate trust with test user u
    sum_of_num = 0.0
    sum_of_den = 0.0
    for user_and_rating in users_and_ratings:
        trust_uv = compute_implicit_trust(u, user_and_rating[0], alpha, beta, sigma, user2item, item2user)
        sum_of_num = sum_of_num + trust_uv * user_and_rating[1]
        sum_of_den = sum_of_den + trust_uv
    if sum_of_den == 0.0:
        return 0.0
    return sum_of_num / sum_of_den

# Run inference on test set using trust-based mean rating
run_inference_on_test_set(test_data, user2item, item2user, calc_trust_based_mean_rating, alpha, beta, sigma)

4088
Item cold start count : 3268
User cold start count : 18378
Coverage : 0.11207061984264056
Mean Absolute Error : 0.9513092351459769
RMSE : 1.8179794690418578


# STEP 6.2 - Method 2 - Trust-based collaborative filtering

In [10]:
def calc_average_user_rating(u, user2item):
    if not u in user2item:
        return 0.0
    return np.average([tup[1] for tup in user2item[u]])


def calc_trust_based_collab_filtering(u, i, alpha, beta, sigma, user2item, item2user):
    """
    Calculate trust-based collaborative filtering rating on test set
    """
    # Get all users who have rated item
    users_and_ratings = [tup for tup in item2user[i]]
    
    # For each user, calculate trust with test user u
    sum_of_num = 0.0
    sum_of_den = 0.0
    for user_and_rating in users_and_ratings:
        trust_uv = compute_implicit_trust(u, user_and_rating[0], alpha, beta, sigma, user2item, item2user)
        sum_of_num = sum_of_num + trust_uv * (user_and_rating[1] - calc_average_user_rating(user_and_rating[0], user2item))
        sum_of_den = sum_of_den + trust_uv
    # Return 0 if no user has rated item
    if sum_of_den == 0.0:
        return 0.0
    return calc_average_user_rating(u, user2item) + (sum_of_num/sum_of_den)

# Run inference on test set using trust-based collaborative filtreing
run_inference_on_test_set(test_data, user2item, item2user, calc_trust_based_collab_filtering, alpha, beta, sigma)

4088
Item cold start count : 3268
User cold start count : 18378
Coverage : 0.11207061984264056
Mean Absolute Error : 1.0613721153161506
RMSE : 2.006375273520283


# STEP 6.3 - Method 3 - Trust-filtered mean

In [11]:
def calc_trust_filtered_mean_rating(u, i, alpha, beta, sigma, user2item, item2user):
    """
    Calculate trust-filterd mean rating on tst set. We filter for trust > 0
    """
    # Get all users who have rated item
    users_and_ratings = [tup for tup in item2user[i]]
    
    # For each user, calculate trust with test user u
    sum_of_num = 0.0
    trust_cnt = 0
    for user_and_rating in users_and_ratings:
        trust_uv = compute_implicit_trust(u, user_and_rating[0], alpha, beta, sigma, user2item, item2user)
        if trust_uv > 0.0:
            sum_of_num = sum_of_num + user_and_rating[1]
            trust_cnt += 1
    if trust_cnt == 0:
        return 0.0
    return sum_of_num / (1.0 * trust_cnt)

# Run inference on test set using trust-filtered mean
run_inference_on_test_set(test_data, user2item, item2user, calc_trust_filtered_mean_rating, alpha, beta, sigma)

4088
Item cold start count : 3268
User cold start count : 18378
Coverage : 0.11207061984264056
Mean Absolute Error : 0.9376778610565589
RMSE : 1.7134297967067953


# STEP 7 - We notice that the coverage is low for the implicit approach. The reason for this that many users have only 1-2 ratings in the dataset. Thus, these users fall in the cold-start case

# We now focus on filtering the dataset for users with >=4 items rated

In [13]:
# Create a user2item dictionary over the entire dataset, and not just training set
user2item_full = {}
for data_tuple in dataset_tuples:
    if data_tuple[1] in user2item_full:
        user2item_full[data_tuple[1]].add((data_tuple[0], data_tuple[2]))
    else:
        itemrating = set()
        itemrating.add((data_tuple[0], data_tuple[2]))
        user2item_full[data_tuple[1]] = itemrating
        
filtered_dataset_tuples = [data_tuple for data_tuple in dataset_tuples if len(user2item_full[data_tuple[1]]) >= 4]

print("Number of data points in original dataset tuples: {}".format(len(dataset_tuples)))
print("Number of data points in filtered dataset tuples: {}".format(len(filtered_dataset_tuples)))

Number of data points in original dataset tuples: 188477
Number of data points in filtered dataset tuples: 54192


# STEP 8 - Split filtered dataset in train/test using 80:20 split

In [14]:
# Randomly shuffle dataset with fixed seed. Then split into train/test using 80:20 ratio
random.Random(4).shuffle(filtered_dataset_tuples)
filtered_train_data = filtered_dataset_tuples[:43600]
filtered_test_data = filtered_dataset_tuples[43600:]

# Print length of train and test dataset
print("Length of filtered training set: {}".format(len(filtered_train_data)))
print("Length of filtered test set: {}".format(len(filtered_test_data)))

Length of filtered training set: 43600
Length of filtered test set: 10592


# STEP 9 - Construct user2item and item2user dictionaries from filtered training data. Also, retain rating for each value in both dictionaries

In [15]:
# Construct user2item from training data
filtered_user2item = {}
for data_tuple in filtered_train_data:
    if data_tuple[1] in filtered_user2item:
        filtered_user2item[data_tuple[1]].add((data_tuple[0], data_tuple[2]))
    else:
        itemrating = set()
        itemrating.add((data_tuple[0], data_tuple[2]))
        filtered_user2item[data_tuple[1]] = itemrating

# Construct item2user from training data
filtered_item2user = {}
for data_tuple in filtered_train_data:
    if data_tuple[0] in filtered_item2user:
        filtered_item2user[data_tuple[0]].add((data_tuple[1], data_tuple[2]))
    else:
        userrating = set()
        userrating.add((data_tuple[1], data_tuple[2]))
        filtered_item2user[data_tuple[0]] = userrating
        
print("Total number of entries in filtered_user2item: {}".format(len(filtered_user2item)))
print("Total number of entries in filtered_item2user: {}".format(len(filtered_item2user)))
print("Printing 5 random entries in filtered_user2item.. ")
for i, val in enumerate(itertools.islice(filtered_user2item, 5)):
    print(val)
    print(filtered_user2item[val])

Total number of entries in filtered_user2item: 7444
Total number of entries in filtered_item2user: 23985
Printing 5 random entries in filtered_user2item.. 
jenmuse
{('pr-Evenflo_Snugli_Comfort_Vent_Carrier_Baby_Carrier', 5), ('Elmer_s_Go_Paint', 5), ('Tek_Nek_Toys_Play_Learn_II_with_Hat_Assortment', 5), ('Aqua_Leisure_SunSmart_Hideaway_Pool', 5), ('Step_2_Frog_Sand_Box', 5), ('pr-Graco_Car_Seat_Base_8403', 5), ('Battat_Parents_Magazine_Record_A_Voice_Cell_Phone', 5), ('Cozy_Coupe_II_Car', 5), ('Playskool_Step_Start_Walk_N_Ride', 3), ('Illco_Sesame_Street_Musical_Lights_Phone', 1), ('Cosco_Alpha_Omega_Elite_Hammer_Print', 5), ('pr-Graco_LiteRider_7320', 5), ('Fisher_Price_Laugh_Learn_Learning_Table_32724820', 5), ('Kids_II_Baby_Einstein_Discovering_Water_Rocker_Seat', 4), ('Fisher_Price_Laugh_and_Learn_Learning_Piggy_Bank', 5)}
apapage
{('Hasbro_VCamNow_Camcorder', 1), ('2003_Nissan_Pathfinder', 1), ('Hewlett_Packard_7210_OfficeJet_All_In_One_Printer_2', 3), ('Palm_Palm_m105_Handheld__P

# STEP 10 - Run inference using all 3 methods on the filtered dataset

In [16]:
# Run inference on test set using trust-based mean rating
print("Running inference on filtered test set using trust-based mean rating")
run_inference_on_test_set(filtered_test_data, filtered_user2item, filtered_item2user, calc_trust_based_mean_rating, alpha, beta, sigma)

# Run inference on test set using trust-based collaborative filtering
print("Running inference on filtered test set using trust-based collaborative filtering")
run_inference_on_test_set(filtered_test_data, filtered_user2item, filtered_item2user, calc_trust_based_collab_filtering, alpha, beta, sigma)

# Run inference on test set using trust-filtered mean
print("Running inference on filtered test set using trust-filtered mean rating")
run_inference_on_test_set(filtered_test_data, filtered_user2item, filtered_item2user, calc_trust_filtered_mean_rating, alpha, beta, sigma)

Running inference on filtered test set using trust-based mean rating
2563
Item cold start count : 3997
User cold start count : 16
Coverage : 0.2419750755287009
Mean Absolute Error : 0.9219902285441315
RMSE : 1.7278778049155208
Running inference on filtered test set using trust-based collaborative filtering
2563
Item cold start count : 3997
User cold start count : 16
Coverage : 0.2419750755287009
Mean Absolute Error : 0.9920943971783541
RMSE : 1.7545552074832798
Running inference on filtered test set using trust-filtered mean rating
2563
Item cold start count : 3997
User cold start count : 16
Coverage : 0.2419750755287009
Mean Absolute Error : 0.9126201375247562
RMSE : 1.642330271480556
